<a href="https://colab.research.google.com/github/ameasure/colab_tutorials/blob/master/Finetune2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Transfer Learning with Finetune 

https://https://github.com/IndicoDataSolutions/finetune

In [1]:

!pip install -U finetune
#!python -m spacy download en_core_web_sm
#!python -m spacy link en_core_web_sm en
#!pip install tables
#!pip install xlrd
!wget 'https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx'

Requirement already up-to-date: finetune in /usr/local/lib/python3.6/dist-packages (0.6.7)
--2019-06-16 14:41:39--  https://github.com/ameasure/autocoding-class/raw/master/msha.xlsx
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ameasure/autocoding-class/master/msha.xlsx [following]
--2019-06-16 14:41:39--  https://raw.githubusercontent.com/ameasure/autocoding-class/master/msha.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4183086 (4.0M) [application/octet-stream]
Saving to: ‘msha.xlsx.1’

msha.xlsx.1         100%[===================>]   3.99M  --.-KB/s    in 0.06s   

2019-06-16 14:41:40 (

In [22]:
import pandas as pd

df = pd.read_excel('msha.xlsx')
df['ACCIDENT_YEAR'] = df['ACCIDENT_DT'].apply(lambda x: x.year)
df['ACCIDENT_YEAR'].value_counts()
df_train = df[df['ACCIDENT_YEAR'].isin([2010, 2011])][:5000].copy()
df_valid = df[df['ACCIDENT_YEAR'] == 2012][:1000].copy()
print('training rows:', len(df_train))
print('validation rows:', len(df_valid))

training rows: 5000
validation rows: 1000


In [0]:
batch_size = 32
val_size = 320
# validate once every epoch
val_interval = int((len(df_train) - val_size) / batch_size)


In [0]:
from finetune import Classifier

# max_length is the maximum number of words we will use from each narrative
# behind the scenes finetune is setting aside 5% of the training data for validation
model = Classifier(batch_size=batch_size, 
                   max_length=90, 
                   n_epochs=10, 
                   val_size=val_size,
                   val_interval=val_interval,
                   eval_acc=True)
model.fit(df_train['NARRATIVE'], df_train['INJ_BODY_PART'])

I0616 15:25:03.078453 139623342266240 base.py:104] Saving tensorboard output to /tmp/Finetunevd9p53tv
I0616 15:25:03.089778 139623342266240 run_config.py:558] Initializing RunConfig with distribution strategies.
I0616 15:25:03.097967 139623342266240 estimator_training.py:167] Not using Distribute Coordinator.
I0616 15:25:03.107203 139623342266240 estimator.py:209] Using config: {'_model_dir': '/tmp/Finetunevd9p53tv', '_tf_random_seed': 42, '_save_summary_steps': 40, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.one_device_strategy.OneDeviceStrategyV1 object at 0x7efba7157e48>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, 

In [17]:
# generate preditions
df_valid['PREDICTED_PART'] = model.predict(df_valid['NARRATIVE'].values)
# look at a sample
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART']].sample(5)

I0616 15:17:01.940991 139623342266240 run_config.py:558] Initializing RunConfig with distribution strategies.
I0616 15:17:01.945101 139623342266240 estimator_training.py:167] Not using Distribute Coordinator.
I0616 15:17:01.948007 139623342266240 estimator.py:209] Using config: {'_model_dir': '/tmp/Finetuneacao16ya', '_tf_random_seed': 42, '_save_summary_steps': 4, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.one_device_strategy.OneDeviceStrategyV1 object at 0x7efba879bbe0>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efba

,NARRATIVE,INJ_BODY_PART,PREDICTED_PART
2486,Coal rolled out from rib striking employee on left lower leg causing a contusion. ****DID NOT START LOSING TIME UNTIL 3/28/12.****,LOWER LEG/TIBIA/FIBULA,"LEG, NEC"
3452,Employee was using a knife to cut a rope and states that the knife slipped and cut his lt. little finger.,FINGER(S)/THUMB,FINGER(S)/THUMB
3457,"Walking on belt line said too much weight on right leg, slipped and fell on back. Had pain in back and right leg was trying to step up on walk board which was wet and muddy.",BACK (MUSCLES/SPINE/S-CORD/TAILBONE),BACK (MUSCLES/SPINE/S-CORD/TAILBONE)
3106,Employee was welding and a foreign body enter his left eye.,EYE(S) OPTIC NERVE/VISON,EYE(S) OPTIC NERVE/VISON
1984,This accident is still under investigation and my be revised at a later date. Employee had parked rock truck to help crusher crew. When he stepped on the second to last step he slipped and fell to the ground into his back. He did not wish for medical attention at that time. On 1/20/12 he decided to see a doctor.,BACK (MUSCLES/SPINE/S-CORD/TAILBONE),BACK (MUSCLES/SPINE/S-CORD/TAILBONE)


In [18]:
pd.options.display.max_colwidth=500

df_valid['PROB_DICT'] = model.predict_proba(df_valid['NARRATIVE'].values)
df_valid[['NARRATIVE', 'PREDICTED_PART', 'PROB_DICT']].head()

I0616 15:17:20.397660 139623342266240 run_config.py:558] Initializing RunConfig with distribution strategies.
I0616 15:17:20.399591 139623342266240 estimator_training.py:167] Not using Distribute Coordinator.
I0616 15:17:20.401229 139623342266240 estimator.py:209] Using config: {'_model_dir': '/tmp/Finetuneacao16ya', '_tf_random_seed': 42, '_save_summary_steps': 4, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.one_device_strategy.OneDeviceStrategyV1 object at 0x7ef8e428d278>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ef8e

,NARRATIVE,PREDICTED_PART,PROB_DICT
2,"Employee, parked s/c on grade at 16-Block #3 Entry Spad #3868. S/c slid approx. 3' pinning oper. between s/c & rib, employee had set park brake and got off machine to move roof bolter cable.",CHEST (RIBS/BREAST BONE/CHEST ORGNS),"{'ABDOMEN/INTERNAL ORGANS': 0.00573621, 'ANKLE': 0.01592992, 'ARM, MULTIPLE PARTS': 0.0036744166, 'ARM,NEC': 0.023610948, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.028688075, 'BODY SYSTEMS': 0.010014769, 'BRAIN': 0.0017826777, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.14970914, 'EAR(S) EXTERNAL': 0.004547225, 'EAR(S) INTERNAL & HEARING': 0.017132862, 'ELBOW': 0.013581002, 'EYE(S) OPTIC NERVE/VISON': 0.029369, 'FACE, MULTIPLE PARTS': 0.00705281, 'FACE,NEC': 0.017550651, 'FINGER(S)/THUMB': 0.0..."
5,Possible heart attack.,WRIST,"{'ABDOMEN/INTERNAL ORGANS': 0.0030353945, 'ANKLE': 0.028163245, 'ARM, MULTIPLE PARTS': 0.0068870313, 'ARM,NEC': 0.0063550975, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.07759733, 'BODY SYSTEMS': 0.06825458, 'BRAIN': 0.0074825175, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.030243406, 'EAR(S) EXTERNAL': 0.0054424205, 'EAR(S) INTERNAL & HEARING': 0.019127002, 'ELBOW': 0.007861541, 'EYE(S) OPTIC NERVE/VISON': 0.03360956, 'FACE, MULTIPLE PARTS': 0.0026610976, 'FACE,NEC': 0.007249103, 'FINGER(S)/THU..."
8,Employee was cleaning up plant spillage into a fork lift mounted self-dumping hopper. Latch on hopper was not operating properly and employee repeatedly lifted on hopper to unlatch it. Employee reported that his right shoulder hurt from the lifting at end of shift. Placed on light duty with no heavy lifting.,SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),"{'ABDOMEN/INTERNAL ORGANS': 0.0011922031, 'ANKLE': 0.002021297, 'ARM, MULTIPLE PARTS': 0.0014828189, 'ARM,NEC': 0.011509362, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.016539313, 'BODY SYSTEMS': 0.002523225, 'BRAIN': 9.0297006e-05, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.008208341, 'EAR(S) EXTERNAL': 0.0012701373, 'EAR(S) INTERNAL & HEARING': 0.0023347044, 'ELBOW': 0.005745471, 'EYE(S) OPTIC NERVE/VISON': 0.0006973524, 'FACE, MULTIPLE PARTS': 0.0011046284, 'FACE,NEC': 0.0024547742, 'FINGER(..."
12,"Employee was putting a drag on a shuttle car, when he lifted one end and another employee lifted the other end, the drag smashed the end of his middle finger on his left hand. He was taken to the hospital and received 12 stitches in his finger and returned to work.",FINGER(S)/THUMB,"{'ABDOMEN/INTERNAL ORGANS': 0.0002954568, 'ANKLE': 0.0005007093, 'ARM, MULTIPLE PARTS': 6.600239e-05, 'ARM,NEC': 0.00023182888, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.000116921925, 'BODY SYSTEMS': 0.00018003216, 'BRAIN': 0.0001448402, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.00015073658, 'EAR(S) EXTERNAL': 0.00019309383, 'EAR(S) INTERNAL & HEARING': 0.00010306011, 'ELBOW': 0.00010767419, 'EYE(S) OPTIC NERVE/VISON': 0.00024160706, 'FACE, MULTIPLE PARTS': 0.0001329841, 'FACE,NEC': 0.000294..."
27,While using a cutting torch to remove a bearing the employee was burned by the torch tip after another employee tripped over the torch hose causing the torch flame to contact his left hand.,HAND (NOT WRIST OR FINGERS),"{'ABDOMEN/INTERNAL ORGANS': 0.00068712817, 'ANKLE': 0.0032244911, 'ARM, MULTIPLE PARTS': 0.00022975702, 'ARM,NEC': 0.0017648677, 'BACK (MUSCLES/SPINE/S-CORD/TAILBONE)': 0.0012839907, 'BODY SYSTEMS': 0.0011765488, 'BRAIN': 0.0003911106, 'CHEST (RIBS/BREAST BONE/CHEST ORGNS)': 0.0021051844, 'EAR(S) EXTERNAL': 0.00036271216, 'EAR(S) INTERNAL & HEARING': 0.0004776119, 'ELBOW': 0.0043536937, 'EYE(S) OPTIC NERVE/VISON': 0.003159223, 'FACE, MULTIPLE PARTS': 0.0005239041, 'FACE,NEC': 0.000595192, 'F..."


In [19]:
# function that takes a row of our dataframe and returns the predicted probability
def get_probability(row):
    predicted_part = row['PREDICTED_PART']
    probability_dict = row['PROB_DICT']
    return probability_dict[predicted_part]

# apply get_probability to each row in our dataframe and store the result
df_valid['PREDICTED_PROB'] = df_valid.apply(func=get_probability, axis=1)
# take a peak at what we get
df_valid[['NARRATIVE', 'INJ_BODY_PART', 'PREDICTED_PART', 'PREDICTED_PROB']].sample(5).head()

,NARRATIVE,INJ_BODY_PART,PREDICTED_PART,PREDICTED_PROB
421,Hearing Loss,EAR(S) INTERNAL & HEARING,EAR(S) INTERNAL & HEARING,0.082944
3249,Employee was performing routine maintenance on the propel motor on shovel 4 when he experienced electric shock.,BODY SYSTEMS,BACK (MUSCLES/SPINE/S-CORD/TAILBONE),0.539016
463,A PIECE OF WATER LINE FELL TWO FEET AND STRUCK EMPLOYEE ON THE SHOULDER CAUSING BRUISING.,SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),SHOULDERS (COLLARBONE/CLAVICLE/SCAPULA),0.903938
2703,Operator was changing drill steel bit when the chain wrench broke and a piece of the bar hit the back/side of the operators left knee. No initial treatment was sought. Area around the impact began to swell after a couple of days and that's when the operator sought medical attention,KNEE/PATELLA,KNEE/PATELLA,0.906240
3402,Employee slipped and fell into a sump hole injuring shoulder and back. Company was notified this was a medical injury on 12/19/2012.,BACK (MUSCLES/SPINE/S-CORD/TAILBONE),MULTIPLE PARTS (MORE THAN ONE MAJOR),0.553728


In [20]:
from sklearn.metrics import accuracy_score, f1_score

mf1 = f1_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'], average='macro')
acc = accuracy_score(df_valid['INJ_BODY_PART'], df_valid['PREDICTED_PART'])
print('macro-f1:', mf1)
print('accuracy:', acc)

macro-f1: 0.33994032603839
accuracy: 0.687


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
